In [1]:
import math

In [2]:
lim_esc = 460* 10**6
lim_ruptura = 550* 10**6
potencia = 120
n = 2000
momento_max = 600
momento_min = -600
forca_axial = 74* 10**3
rasgo_chaveta = 'sim'
L = 1200 * 10**-3
tipo_choque = 'moderado'
k = 0.6
d= ''


In [3]:
#determinação da potência, das rotações em rad/s, torque máximo 
pot = potencia * 745.7

N = n * 0.1047

torque_max = round(pot / N ,2)


In [4]:
#determinação do km e kt
if tipo_choque == 'moderado':
  km = 2
  kt = 1.5
elif tipo_choque == 'gradual':
  km = 1.5
  kt = 1
elif tipo_choque == 'forte':
  km = 3
  kt = 3

In [5]:
#derterminação da tensão admissível
if rasgo_chaveta == 'sim':
  x = 0.225 * lim_esc
  y = 0.135 * lim_ruptura
  if x < y:
    tensao_adm = x
  else:
    tensao_adm = y
else:
  x = 0.3 * lim_esc
  y = 0.18 * lim_ruptura
  if x < y:
    tensao_adm = x
  else:
    tensao_adm = y

In [6]:
#valor inicial do diâmetro externo
d0 = ((32/(3.14*tensao_adm*(1-(k**4))))*(momento_max**2 + (3/4*(torque_max**2)))**(1/2))**(1/3)
print(d0)

0.04808274353442453


In [7]:
#fórmula utilizada para encontrar o diâmetro no método de newton
def f(d):
  return d**3 - ((32/(3.14*tensao_adm*(1-(k**4))))* ((km*momento_max + (1/8 * (1 / (1 - 0.0044*(L/((1/4 * d * math.sqrt(1+ 0.6**2)))))) * forca_axial * d * (1+ k**2)))**2 + (3/4*(kt*torque_max**2)))**(1/2))


In [8]:
#encontrar o valor da derivada
def flin(f, d):  
  if d == 0:
    d = 1
    h = d / 5471
    flin = (1/(12*h)) * ( f(d- 2*h) - 8*f(d - h) + 8*f(d+h) - f(d + 2*h) )
  else:
    h = d / 5471
    flin = (1/(12*h)) * ( f(d- 2*h) - 8*f(d - h) + 8*f(d+h) - f(d + 2*h) )
  return flin

In [9]:
def newton(f, d0, epsilon, maxIter=20):
    if abs(f(d0)) <= epsilon:
      return (d0)

    print("k\t d0\t\t f(d0)") #cabeçalho da tabela que retornará com os valores embaixo. \t é o espaçamento que dou para ficar visual.
    
    k=1
    while k <= maxIter:
      d1 = d0 - f(d0)/flin(f,d0)
        
      print("%d\t%e\t%e" % (k, d1,f(d1)))

      if abs(f(d1)) <= epsilon:
        return d1      
      
      d0 = d1
      k = k + 1
    
    print('ERRO: Número máximo de iterações atingido')
    
    return d1  

In [10]:
#determinando o diâmetro utilizando o método de newton
d = newton(f, d0, 10**-8, maxIter=20)

print(d)

print(f'Diâmetro externo : {(d*10**3):.3f} mm')

#diâmetro interno
d_int = k*d*10**3
print(f'Diâmetro interno: {d_int:.2f} mm')

k	 d0		 f(d0)
1	8.989699e-02	3.078035e-04
2	7.616446e-02	4.784186e-05
3	7.310593e-02	2.075312e-06
4	7.296075e-02	4.535289e-09
0.07296074853442176
Diâmetro externo : 72.961 mm
Diâmetro interno: 43.78 mm


In [11]:

def_ang = (583.61 * torque_max)/ ((80*10**9) * (d**4) * (1 - (k**4)))
print(f'Deflexão angular: {def_ang:.2f}°/m')

Deflexão angular: 0.13°/m
